<a href="https://colab.research.google.com/github/jpmantuano/csc612m/blob/main/Discovery_Series_01_Understanding_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Joseph Paulo L Mantuano

## 1: C/C++ program version

In [ ]:
%%writefile convolve.cpp
#include <iostream>
#include <vector>
#include <ctime>
#include <cstdio>

void convolve1D(const std::vector<float>& in, std::vector<float>& out) {
    int n = in.size();
    if (n < 3) {
        out.clear();
        return;
    }
    out.resize(n - 2);
    for (int i = 0; i < n - 2; ++i) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

int main() {
    int arr[] = {
      256,
      1024,
      16777216, //2^24
      67108864, //2^26
      268435456 //2^28
    };

    int length = sizeof(arr) / sizeof(arr[0]);

    for (int j = 0; j < length; j++) {

        std::vector<float> _in_(arr[j]);

        for (int i = 0; i < arr[j]; ++i) {
            _in_[i] = static_cast<float>(i + 1);
        }
        std::vector<float> _out_;

      clock_t start, end;
      double cpu_time_used;
      double sum = 0.0;

      for (int run = 0; run < 10; run++) {
        start = clock();  // Start time
        convolve1D(_in_, _out_);
        end = clock(); // End time

        cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
        sum += cpu_time_used;
      }

      double avg_time = sum / 10;
      printf("Average execution time for %d inputs: %.6f seconds\n", arr[j], avg_time);
    }

    return 0;
}

In [ ]:
%%shell
g++ -o convolve convolve.cpp

In [ ]:
%%shell
./convolve

### CPU execution time (at least ten runs): ###

Execution time in milliseconds.  Multiple kernel run is based on average execution time.

| Number of elements | Avg. execution time |
| ------------------ | ------------------- |
| 256                | 0.005 ms            |
| 1024               | 0.017 ms            |
| 2^24               | 191.308 ms          |
| 2^26               | 843.958 ms          |
| 2^28               | 3345.247 ms         |

## 2: CUDA program version using grid-stride loop without prefetch

In [ ]:
%%writefile CUDA_convolve.cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <vector>

__global__ void conv1d_kernel(const float* in, float* out, int n) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = idx; i < n - 2; i += stride) {
      out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
  }
}

int main(int argc, char* argv[]) {
    if (argc < 2) {
        std::cerr << "Usage: " << argv[0] << " <N>" << std::endl;
        return 1;
    }

    int N = std::atoi(argv[1]);
    if (N <= 0) {
        std::cerr << "N must be a positive integer." << std::endl;
        return 1;
    }

    // const int N = 256;

    // Use vector for dynamic size and easy modification
    std::vector<float> h_in(N);
    std::vector<float> h_out(N, 0.0f);

    // Initialize h_in with values 1.0, 2.0, 3.0, ..., N
    for (int i = 0; i < N; ++i) {
        h_in[i] = static_cast<float>(i + 1);
    }

    float *d_in = nullptr, *d_out = nullptr;
    cudaMalloc(&d_in, N * sizeof(float));
    cudaMalloc(&d_out, N * sizeof(float));

    cudaMemcpy(d_in, h_in.data(), N * sizeof(float), cudaMemcpyHostToDevice);

    int threadsPerBlock = 1024;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
//    int threadsPerBlock = (N < 1024) ? N : 1024;
//    int blocksPerGrid = 1;

    conv1d_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_in, d_out, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_out.data(), d_out, N * sizeof(float), cudaMemcpyDeviceToHost);

//    std::cout << "Convolution output:\n";
//    for (int i = 0; i < N - 2; ++i) {
//        std::cout << h_out[i] << " ";
//    }
//    std::cout << std::endl;

    cudaFree(d_in);
    cudaFree(d_out);

    return 0;
}

In [ ]:
%%shell
nvcc CUDA_convolve.cu -o CUDA_convolve -arch=sm_75

In [ ]:
%%shell
./CUDA_convolve 16777216

In [ ]:
%%shell
nvprof --unified-memory-profiling per-process-device --print-gpu-trace ./CUDA_convolve 16777216

In [ ]:
%%shell
nvprof ./CUDA_convolve 16777216

In [ ]:
%%shell
for i in {1..10}
do
  nvprof ./CUDA_convolve 16777216 2>&1 | tee run_$i.log
done

### Execution time in milliseconds.  Multiple kernel run is based on average execution time. ###

a.) Number of blocks/grid = 1

| Block size (2^24 elements) | Single kernel run | Multiple kernel run |
| -------------------------- | ----------------- | ------------------- |
| Block size = 1024          | 12.633ms          | 12.6288ms           |

b.) Number of blocks/grid = max (as per formula)

| Block size (2^24 elements) | Single kernel run | multiple kernel run |
| -------------------------- | ----------------- | ------------------- |
| Block size = 1024          | 1.1660 ms         | 1.1668 ms           |

c.) Unified memory profile - data transfer (2^24 elements)

| Type           | Total size | Total time |
| -------------- | ---------- | ---------- |
| host to device | 64 MB      | 14.217ms   |
| device to host | 64 MB      | 14.899ms   |

d.) Unified memory profile - page fault (2^24 elements)

| Type                 | Count | Total time |
| -------------------- | ----- | ---------- |
| GPU page fault group |       |            |
| CPU Page fault group |       | -----      |

a.) What is the effect of having one block only vs. max block in terms of execution time?  Why is it so?
- The number of blocks allows the increase parallelism and utilization of GPU resources. Having one block, all executions are processed in a single GPU processor while having max blocks allows the exploit of maximizing GPU processors.

b.) What is the effect of single run vs multiple run in terms of execution time? Why is it so?
- The average execution time decreases if taken after multiple runs vs a single run. Because in a multiple run there is a cache effect after transferring the data after the first run making the succeeding runs faster because of lower overhead compared to a single run.

c.) In the unified memory profile, is the data transfer time included in the GPU execution time?  Explain your answer.
- the data transfer time is generally included in the GPU execution time reported by the profiler. The amount of time taken to move data from host to device and device to host for processing by the GPU.

d.)  In the unified memory profile, is the page fault time included in the GPU execution time?  Explain your answer.
- No, the code uses cudaMalloc which does explicit memory allocations and not unified memory.

e.) What is the speedup (or speed down) of the execution time of GPU (including all the overhead) compare to C/C++? Is GPU execution time better or worse?
- In general the GPU execution is faster, even on the one block runs. But when running with smaller number of elements for computation the difference between GPU runtime vs CPU runtime is smaller. Probably due to the amount of processing needed with smaller data, the advantage of using GPU for processing decreases.

## 3: CUDA Program Version Using Grid-Stride Loop With Prefetching

In [ ]:
%%writefile CUDA_convolve_prefetch.cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>

__global__ void conv1d_kernel(const float* __restrict__ in, float* __restrict__ out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = idx; i < n - 2; i += stride) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

int main(int argc, char* argv[]) {
    if (argc < 2) {
        std::cerr << "Usage: " << argv[0] << " <N>" << std::endl;
        return 1;
    }

    int N = std::atoi(argv[1]);
    if (N <= 0) {
        std::cerr << "N must be a positive integer." << std::endl;
        return 1;
    }

    // const int N = 256;

    float *h_in = nullptr;
    float *h_out = nullptr;

    // Allocate unified memory (managed memory)
    cudaMallocManaged(&h_in, N * sizeof(float));
    cudaMallocManaged(&h_out, N * sizeof(float));

    // Initialize input
    for (int i = 0; i < N; ++i) {
        h_in[i] = float(i + 1);
        h_out[i] = 0.0f;
    }

    // Prefetch input and output to GPU
    int device = -1;
    cudaGetDevice(&device);
    cudaMemPrefetchAsync(h_in, N * sizeof(float), device);
    cudaMemPrefetchAsync(h_out, N * sizeof(float), device);

    int threadsPerBlock = 1024;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
//    int threadsPerBlock = (N < 1024) ? N : 1024;
//    int blocksPerGrid = 1;

    conv1d_kernel<<<blocksPerGrid, threadsPerBlock>>>(h_in, h_out, N);
    cudaDeviceSynchronize();

    // Prefetch output back to CPU (optional)
    cudaMemPrefetchAsync(h_out, N * sizeof(float), cudaCpuDeviceId);

    cudaDeviceSynchronize();

//    std::cout << "Convolution output:\n";
//    for (int i = 0; i < N - 2; ++i) {
//        std::cout << h_out[i] << " ";
//    }
//    std::cout << std::endl;

    cudaFree(h_in);
    cudaFree(h_out);

    return 0;
}

In [ ]:
%%shell
nvcc CUDA_convolve_prefetch.cu -o CUDA_convolve_prefetch -arch=sm_75

In [ ]:
%%shell
./CUDA_convolve_prefetch 16777216

In [ ]:
%%shell
nvprof --unified-memory-profiling per-process-device --print-gpu-trace ./CUDA_convolve_prefetch 16777216

In [ ]:
%%shell
nvprof ./CUDA_convolve_prefetch 16777216

In [ ]:
%%shell
for i in {1..10}
do
  nvprof ./CUDA_convolve_prefetch 16777216 2>&1 | tee run_$i.log
done

### Execution time in milliseconds.  Multiple kernel run is based on average execution time. ###

a.) Number of blocks/grid = max (as per formula)

| Block size (2^24 elements) | Single kernel run | multiple kernel |
| -------------------------- | ----------------- | --------------- |
| Block size = 1024          | 1.1652ms          | 1.16742ms       |

b.)Unified memory profile - data transfer (2^24 elements)

| Type           | Total size | Total time |
| -------------- | ---------- | ---------- |
| host to device | 128 MB     | 11.13964ms |
| device to host | 64 MB      | 5.142931ms |

c.) Unified memory profile - page fault (2^24 elements)

| Type                 | Count | Total time |
| -------------------- | ----- | ---------- |
| GPU page fault group |       |            |
| CPU Page fault group | 384   | ---------  |

a.) In the unified memory profile, is the data transfer time included in the GPU execution time?  Explain your answer.
- the data transfer time is generally included in the GPU execution time reported by the profiler. The amount of time taken to move data from host to device and device to host for processing by the GPU.

b.) In the unified memory profile, is the page fault time included in the GPU execution time?  Explain your answer.
- Yes, page faults is included in the profile results. Using cudaMallocManaged, which makes a unified address for both CPU and GPU, when the GPU accesses a memory page not currently resident in its local memory, a page fault occurs.

c.) What is the speedup (or speed down) of the execution time of GPU with prefetching compare to without prefetching?  Is GPU execution time (with prefetching) better or worse? Include all the overhead in the computation.
- There is a speed up around ~ 12ms with prefetching. Probably due to the data transfer not blocking the kernel processing time, since the data was already transfer prior to processing.

d.) For this case, is there an effect in execution time if kernel is executed multiple times as compare to executing once only? Why is it so?
- The runs shows very small difference in runtimes. Probably because if using prefetching, data is already moved to the GPU, it reduces the overall impact in processing / kernel execution.

## 4: CUDA Program Version Using Grid-Stride Loop With Prefetch and Memory Advice

In [ ]:
%%writefile CUDA_convolve_prefetch_mem_advice.cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>

__global__ void conv1d_kernel(const float* __restrict__ in, float* __restrict__ out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = idx; i < n - 2; i += stride) {
        out[i] = (in[i] + in[i + 1] + in[i + 2]) / 3.0f;
    }
}

int main(int argc, char* argv[]) {
    if (argc < 2) {
        std::cerr << "Usage: " << argv[0] << " <N>" << std::endl;
        return 1;
    }

    int N = std::atoi(argv[1]);
    if (N <= 0) {
        std::cerr << "N must be a positive integer." << std::endl;
        return 1;
    }

    //const int N = 256;

    float *h_in = nullptr;
    float *h_out = nullptr;

    // Allocate unified memory
    cudaMallocManaged(&h_in, N * sizeof(float));
    cudaMallocManaged(&h_out, N * sizeof(float));

    // Initialize input
    for (int i = 0; i < N; ++i) {
        h_in[i] = float(i + 1);
        h_out[i] = 0.0f;
    }

    int device = -1;
    cudaGetDevice(&device);

    // Advise that h_in will be mostly read on device
    cudaMemAdvise(h_in, N * sizeof(float), cudaMemAdviseSetReadMostly, device);
    // Advise that h_out will be mostly written on device
    cudaMemAdvise(h_out, N * sizeof(float), cudaMemAdviseSetPreferredLocation, device);

    // Prefetch to device
    cudaMemPrefetchAsync(h_in, N * sizeof(float), device);
    cudaMemPrefetchAsync(h_out, N * sizeof(float), device);

    int threadsPerBlock = 1024;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
//    int threadsPerBlock = (N < 1024) ? N : 1024;
//    int blocksPerGrid = 1;

    conv1d_kernel<<<blocksPerGrid, threadsPerBlock>>>(h_in, h_out, N);
    cudaDeviceSynchronize();

    // Prefetch output back to host
    cudaMemPrefetchAsync(h_out, N * sizeof(float), cudaCpuDeviceId);
    cudaDeviceSynchronize();

//    std::cout << "Convolution output:\n";
//    for (int i = 0; i < N - 2; ++i) {
//        std::cout << h_out[i] << " ";
//    }
//    std::cout << std::endl;

    cudaFree(h_in);
    cudaFree(h_out);

    return 0;
}

In [ ]:
%%shell
nvcc CUDA_convolve_prefetch_mem_advice.cu -o CUDA_convolve_prefetch_mem_advice -arch=sm_75

In [ ]:
%%shell
./CUDA_convolve_prefetch_mem_advice 256

In [ ]:
%%shell
nvprof --unified-memory-profiling per-process-device --print-gpu-trace ./CUDA_convolve_prefetch_mem_advice 16777216

In [ ]:
%%shell
nvprof ./CUDA_convolve_prefetch_mem_advice 268435456

In [ ]:
%%shell
for i in {1..10}
do
  nvprof ./CUDA_convolve_prefetch_mem_advice 268435456 2>&1 | tee run_$i.log
done

### Execution time in milliseconds.  Multiple kernel run is based on average execution time. ###

a.) Number of blocks/grid = max (as per formula)

| Block size (2^24 elements) | Single kernel run | multiple kernel |
| -------------------------- | ----------------- | --------------- |
| Block size = 1024          | 1.1690ms          | 1.16706ms       |

b.)Unified memory profile - data transfer (2^24 elements)

| Type           | Total size | Total time |
| -------------- | ---------- | ---------- |
| host to device | 128 MB     | 11.14467ms |
| device to host | 64 MB      | 5.151089ms |

c.) Unified memory profile - page fault (2^24 elements)

| Type                 | Count | Total time |
| -------------------- | ----- | ---------- |
| GPU page fault group |       |            |
| CPU Page fault group | 384   | --------   |

a.) In the unified memory profile, is the data transfer time included in the GPU execution time?  Explain your answer.

b.) What is the speedup (or speed down) of the execution time of GPU (with prefetching, page creation and memadvise) compare to execution time of C?  Is GPU execution time better or worse? Include all the overhead in the computation.

c.) For this case, is there an effect in execution time if kernel is executed multiple times as compare to executing once only? Why is it so?

### Execution time in milliseconds.  Multiple kernel run is based on average execution time ###

Grid-stride loop (prefetching with "page creation" and mem advise)

### block size = 256 threads ###

| # of elements | Average based on multiple  kernel run | Speedup compare to C | Speedup compare to single run (no prefetch) |
| ------------- | ------------------------------------- | -------------------- | ------------------------------------------- |
| 256           | 88.448 us                             | 0.005 ms             | 7.264 us                                    |
| 1024          | 12.608 us                             | 0.017 ms             | 7.456 us                                    |
| 2^24          | 17.168322 ms                          | 191.308 ms           | 30.16546 ms                                 |
| 2^26          | 68.26708 ms                           | 843.958 ms           | 118.8253 ms                                 |
| 2^28          | 269.20204 ms                          | 3345.247 ms          | 474.072 ms                                  |

##### 2^24 [16777216] elements #####

| Block size (2^24 elements) | Single kernel run | multiple kernel run |
| -------------------------- | ----------------- | ------------------- |
| 256                        | 17.161082 ms      | 17.166806 ms        |
| 512                        | 17.157936 ms      | 17.155219 ms        |
| 1024                       | 17.168466 ms      | 17.156158 ms        |

##### 2^26 [67108864] elements #####

| Block size (2^26 elements) | Single kernel run | multiple kernel run |
| -------------------------- | ----------------- | ------------------- |
| 256                        | 68.67913 ms       | 67.85576 ms         |
| 512                        | 69.0517 ms        | 68.09373 ms         |
| 1024                       | 69.22053 ms       | 67.18557 ms         |

##### 2^28 [268435456] elements #####

| Block size (2^28 elements) | Single kernel run | multiple kernel run |
| -------------------------- | ----------------- | ------------------- |
| 256                        | 275.75955 ms      | 273.18701 ms        |
| 512                        | 270.15548 ms      | 271.33376 ms        |
| 1024                       | 275.72904 ms      | 269.23974 ms        |

a.) What is the effect of the number of elements in the execution time?  What is the rate of increase (i.e., linear, logarithmic, exponential, etc.)?
- the number of elements increases the runtime. Across all the runs, the increase in runtimes are mostly linear with outlier in some occasions. The kernel session was restarted every set of runs to ensure a clean environment.

b.) How does block size affect execution time (observing various elements and using max blocks)?  Which block size will you recommend?
- there is a slight improvements in runtime as the block size is being increased, with outliers in the single kernel run. But overall, the improvement in runtime for multiple kernel runs seems to improve or is within range of the runtime from the other block sizes. This might be due to the size of the elements being tested. More tests with a larger number of elements can prove if the outliers is due to a small number of elements during the test.

c.) Is prefetching always recommended?  Can you think of a situation in which no prefetching is better?
- In general prefetching is recommended, because it can help to reduce processing overhead, by moving data to the GPU prior to being accessed. But similar to caching it may sometimes bring unnecessary data transfer to the GPU causing traffic and not bringing performance gain. Or when the data is small enough and processing time is short enough, the overhead of prefetching does not bring any significant gain.